# Part 2: Water–Gas Shift Reaction

## Introduction

In this notebook, you will apply the PySCF RRHO workflow from **Part 1** to a chemically meaningful equilibrium:
the **water–gas shift (WGS) reaction**

$$
\mathrm{CO + H_2O \rightleftharpoons CO_2 + H_2}.
$$

The focus in Part 2 is on assembling *reaction* thermodynamics from *molecular* thermochemistry. You will compute
RRHO Gibbs free energies for each species and combine them to obtain:

- the standard reaction Gibbs free energy, $\Delta_r G^\circ(T)$, and
- the corresponding equilibrium constant, $K^\circ(T)$.

## Learning goals

After completing Part 2, you should be able to:

- compute RRHO thermochemistry for multiple species at a consistent level of theory,
- extract $G^\circ$ from each `thermo_info` dictionary (introduced in Part 1),
- assemble $\Delta_r G^\circ$ for a reaction and convert it to $K^\circ$, and
- compare gas-phase results to a PCM (implicit-solvent) variant and interpret the qualitative shift.

:::{note} Conventions
Unless stated otherwise, we use $T = 298.15\,\mathrm{K}$ and $P = 1\,\mathrm{bar}$.
Pressure is passed to PySCF in Pa, so $1\,\mathrm{bar} = 100000\,\mathrm{Pa}$.
All species should be treated with the same level of theory and basis set.
- Level of theory: `wB97X-V` / `def2-QZVP`.
:::


In [ ]:
from pyscf import dft, gto
from pyscf.geomopt.geometric_solver import optimize
from pyscf.hessian import thermo

In [ ]:
import patch

## Molecules for the water–gas shift reaction

Define `gto.M(...)` objects for **CO**, **H₂O**, **CO₂**, and **H₂**.

:::{tip} Geometry input
Use simple, reasonable starting geometries. The helper function below optimizes each structure before
computing thermochemistry, so the initial coordinates only need to be plausible.
:::


In [ ]:
co = gto.M(
    atom="""
    C 0.0000 0.0000 0.0000
    O 0.0000 0.0000 1.13
    """,
    basis="def2-QZVP",
    verbose=3,
)

# TODO: Define h2o, co2, and h2 with the same basis set.

## Helper function: RRHO thermochemistry for one molecule

To avoid repeating the same sequence of PySCF calls, we use a helper function that:

1. runs a geometry optimization,
2. computes a Hessian and harmonic frequencies, and
3. returns the `thermo_info` dictionary from `thermo.thermo(...)`.

:::{note}
This reproduces the workflow introduced in Part 1. The key idea is that each species produces its own
`thermo_info` dictionary, from which we will extract $G^\circ$ using the key `"G_tot"`.
:::


In [ ]:
def compute_thermo_for_molecule(mol, T=298.15, P=100000.0, use_pcm=False):
    """Run geometry optimization and RRHO thermochemistry for a molecule.

    Parameters
    ----------
    mol : pyscf.gto.Mole
        Molecule object.
    T : float
        Temperature in K.
    P : float
        Pressure in Pa.
    use_pcm : bool
        If True, include PCM water.

    Returns
    -------
    dict
        Thermochemistry dictionary from thermo.thermo.
    """

    mf = dft.RKS(mol)
    mf.xc = "wB97X-V"

    if use_pcm:
        mf = mf.PCM()
        mf.with_solvent.eps = 78.3553

    mol_opt = optimize(mf)

    mf_opt = dft.RKS(mol_opt)
    mf_opt.xc = "wB97X-V"
    if use_pcm:
        mf_opt = mf_opt.PCM()
        mf_opt.with_solvent.eps = 78.3553

    _ = mf_opt.kernel()

    hess_opt = mf_opt.Hessian().kernel()
    freq_info = thermo.harmonic_analysis(mol_opt, hess_opt)

    thermo_info = thermo.thermo(mf_opt, freq_info["freq_au"], T, P)
    return thermo_info

## Gas-phase reaction thermochemistry

1. Compute thermochemistry for each species in the **gas phase** (`use_pcm=False`).
2. Extract the total Gibbs free energy using the key `"G_tot"` (a `(value, unit)` pair).
3. Assemble
   
   $$
   \Delta_r G^\circ(T) =
   G^\circ_{\mathrm{CO_2}}(T) + G^\circ_{\mathrm{H_2}}(T)
   - G^\circ_{\mathrm{CO}}(T) - G^\circ_{\mathrm{H_2O}}(T).
   $$

4. Convert to an equilibrium constant via
   $$
   K^\circ(T) = \exp\bigl(-\Delta_r G^\circ(T)/(RT)\bigr).
   $$

:::{admonition} Checkpoint
Before moving on, verify that you used the **same** level of theory and basis set for all four species.
:::


In [ ]:
import math

T = 298.15  # K
P = 100000.0  # Pa (1 bar)
R = 0.008314462618  # kkJ mol^-1 K^-1
hartree_to_kjmol = 2625.499748

thermo_co_gas = compute_thermo_for_molecule(co, T=T, P=P, use_pcm=False)
# TODO: thermo_h2o_gas = ...
# TODO: thermo_co2_gas = ...
# TODO: thermo_h2_gas  = ...

In [ ]:
G_key = "G_tot"

G_co_gas, _unit = thermo_co_gas[G_key]
# TODO: G_h2o_gas, _ = ...
# TODO: G_co2_gas, _ = ...
# TODO: G_h2_gas, _  = ...

delta_G_gas = (G_co2_gas + G_h2_gas) - (G_co_gas + G_h2o_gas)
delta_G_gas_kJmol = delta_G_gas * hartree_to_kjmol

K_gas = math.exp(-delta_G_gas_kJmol / (R * T))
print(f"Gas-phase Δ_r G°(T={T} K) = {delta_G_gas_kJmol:.2f} kJ/mol")
print(f"Gas-phase K°(T={T} K)      = {K_gas:.3e}")

## Including solvent effects (PCM)

Repeat the analysis with a PCM water model (`use_pcm=True`) and compare the results.

Conceptually, PCM adds an approximate solvation free energy to each species. The reaction free energy then
reflects differences in solvation between reactants and products.

:::{admonition} Discussion prompt
Does PCM shift the equilibrium toward reactants or products at 298.15 K?
Which term (CO/CO₂ vs H₂O/H₂) appears most responsible for the shift?
:::


In [ ]:
thermo_co_pcm = compute_thermo_for_molecule(co, T=T, P=P, use_pcm=True)
# TODO: thermo_h2o_pcm = ...
# TODO: thermo_co2_pcm = ...
# TODO: thermo_h2_pcm  = ...

G_co_pcm, _unit = thermo_co_pcm[G_key]
# TODO: G_h2o_pcm, _ = ...
# TODO: G_co2_pcm, _ = ...
# TODO: G_h2_pcm, _  = ...

delta_G_pcm = (G_co2_pcm + G_h2_pcm) - (G_co_pcm + G_h2o_pcm)
delta_G_pcm_kJmol = delta_G_pcm * hartree_to_kjmol

K_pcm = math.exp(-delta_G_pcm_kJmol / (R * T))
print(f"PCM Δ_r G°(T={T} K) = {delta_G_pcm_kJmol:.2f} kJ/mol")
print(f"PCM K°(T={T} K)      = {K_pcm:.3e}")